###### Utils

In [ ]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), '../src'))
sys.path.append(os.path.join(os.getcwd(), './data'))
import csv
import numpy as np, matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 6]
from datetime import datetime
import calendar

from arch import arch_model

from MultiValidPrediction import MultiValidPrediction

In [ ]:
def stock_history(name, datefrom='01/03/00', dateto='12/31/20', ret_scale=100):
  file = open('.\data\\' + name + '.csv')
  csv_file = csv.reader(file)
  header= next(csv_file)

  rows = []
  for row in csv_file:
          rows.append(row)

  rows = np.array(rows)

  dates = np.array(rows[:, 0])
  open_prices = np.array([float(price) for price in rows[:, 1]])

  begin = np.where(dates==datefrom)[0][0]

  end = np.where(dates==dateto)[0][0]

  prices = open_prices[end:begin][::-1]

  T = len(prices)

  returns = [(prices[1]/prices[0]-1)]
  for t in range(1, T):
    returns.append(prices[t]/prices[t-1] - 1)
  
  returns = [ret * ret_scale for ret in returns] # scale returns

  volatility = [ret**2 for ret in returns]

  f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=[15, 12], sharex=True)
  ax1.plot(prices)
  ax1.set_ylabel(name + ' prices')
  ax2.plot(returns)
  ax2.set_ylabel(name + ' returns')
  ax3.plot(volatility)
  ax3.set_ylabel(name + ' volatility')
  f.suptitle(name + ' Historical Data', fontsize=20)
  f.show()
  
  return T, prices, returns, volatility

In [ ]:
def garch_scores(returns, volatility, lookback=100, offset=10, score_type='regr', score_unit_interval=True, norm_const=-1):
  # initialize prediction model
  garch_model = arch_model(returns, vol='Garch', p=1, q=1)

  predictions = [0 for i in range(offset)]
  scores = [0 for i in range(offset)]

  for t in range(offset, len(volatility)):
      # current window indices
      left_ind = max(0, t-lookback)
      right_ind = t

      # compute score
      variance_pred_array = garch_model.fit(first_obs=left_ind, last_obs=right_ind, disp='off', show_warning=False).forecast(reindex=False).variance
      varNext = variance_pred_array.iloc[0]['h.1'] #['h.1'][lookback-1]
      score = score_fn(actual=volatility[t], pred=varNext, score_type=score_type, unit_interval_norm=score_unit_interval, divide_by_const=(norm_const != -1), norm_const=norm_const)

      # update arrays with data
      scores.append(score)
      predictions.append(varNext)

  return scores, predictions

def score_fn(actual, pred, score_type, unit_interval_norm=False, divide_by_const=False, norm_const=1000):
  # what kind of score?
  if score_type=='regr':
    scr = abs(actual-pred)
  if score_type=='regr_normalized':
    scr = abs(actual-pred)/pred
  
  # normalize score into [0, 1]
  if unit_interval_norm: 
    if divide_by_const:
      scr /= norm_const
    else:
      scr = scr/(1+scr)
  return scr

In [ ]:
def ACI_method(scores, alpha=0.1, lookback=100, offset=10, gamma=0.005):
  T = len(scores)

  # initialize ACI
  alphat = alpha

  alphas = [alpha for i in range(offset)]
  thresholds = [0 for i in range(offset)]
  err_seq = [0 for i in range(offset)]

  for t in range(offset, T):
      # current scoring window
      left_ind = max(0, t-lookback)
      right_ind = t
      recent_scores = scores[left_ind: right_ind]

      if 1 - alphat > 1:
        threshold = 1
      elif 1 - alphat < 0:
        threshold = 0
      else:
        threshold = np.quantile(recent_scores, 1-alphat)
      err_ind = int(scores[t] > threshold)

      # ACI alphat update
      alphat = alphat + gamma*(alpha-err_ind)

      # update arrays with data
      alphas.append(alphat)
      thresholds.append(threshold)
      err_seq.append(err_ind)
  
  miscoverage_rate_ACI = np.mean(np.array(err_seq))
  print('ACI miscoverage rate: ', miscoverage_rate_ACI)

  return thresholds, alphas, miscoverage_rate_ACI

###### Data processing

In [ ]:
T, prices, returns, volatility = stock_history('AMD', ret_scale=100)

In [ ]:
# CREATE NOISY DATA FOR MULTIGROUP EXPERIMENT

num_groups = 20

returns_noisy = np.zeros(len(returns))
std_returns = np.std(returns)
for t in range(len(returns)):
  returns_noisy[t] = returns[t]
  for j in range(1, num_groups+1):
    if t % j == 0:
      returns_noisy[t] += std_returns*np.random.randn()

volatility_noisy = [ret**2 for ret in returns_noisy]

In [ ]:
scores_noisy, predictions_noisy = garch_scores(returns_noisy, volatility_noisy)

In [ ]:
scores, predictions = garch_scores(returns, volatility)

###### Experiment: Noisy Multigroup Coverage

In [ ]:
# ACI Analysis

ACI_thrs, ACI_alphas, ACI_miscoverage_rate = ACI_method(scores_noisy, alpha=0.1, lookback=100, offset=10, gamma=0.005)

score_seq = scores_noisy
thrs_seq = ACI_thrs
print_ACI_group_stats = False

g_miscoverage = np.zeros(num_groups+1, dtype=int)
g_counts = np.zeros(num_groups+1, dtype=int)

g_counts_residual = 0
g_miscoverage_residual = 0
g_counts_div_by_smth = 0
g_miscoverage_div_by_smth = 0

for t in range(10, len(thrs_seq)):
  divisible_by_something = False
  for j in range(1, num_groups+1):
    if t % j == 0:
      g_counts[j] += 1
      g_miscoverage[j] += int(score_seq[t] > thrs_seq[t])
      if j > 1:
        divisible_by_something = True
  if not divisible_by_something:
    g_counts_residual += 1
    g_miscoverage_residual += int(score_seq[t] > thrs_seq[t])
  else: 
    g_counts_div_by_smth += 1
    g_miscoverage_div_by_smth += int(score_seq[t] > thrs_seq[t])

ACI_coverage_rate = 1-np.array([g_miscoverage[j]/g_counts[j] for j in range(1, num_groups+1)])

if print_ACI_group_stats:
    print('Per-Group Coverage Statistics:')

    for j in range(num_groups-1):
      print('Group ', j+1, ' : Coverage: ', ACI_coverage_rate[j])

    print(1 - g_miscoverage_residual/g_counts_residual)
    print(g_miscoverage_residual)
    print(g_counts_residual)

    print(1 - g_miscoverage_div_by_smth/g_counts_div_by_smth)
    print(g_miscoverage_div_by_smth)
    print(g_counts_div_by_smth)

In [ ]:
# MVP

groups = []
for i in range(1, num_groups + 1):
    group = (lambda t, i=i: (t % i) == 0)
    groups.append(group)

n_buckets = 40
T = len(scores_noisy)
eta = np.sqrt(np.log(len(groups) * n_buckets) / (2 * len(groups) * n_buckets))

MVP_group = MultiValidPrediction(n_buckets=n_buckets, groups=groups, eta=eta)

for t in range(T):
    x_t = t
    score = scores_noisy[t]

    thr = MVP_group.predict(x_t)
    MVP_group.update(x_t, thr, score)

thresholds_MVP_group, miscoverage_MVP_group, counts_MVP_group = MVP_group.coverage_stats(plot_thresholds=False, print_per_group_stats=False)

MVP_coverage_rate = 1-np.array([np.sum(miscoverage_MVP_group[:, j])/np.sum(counts_MVP_group[:, j]) for j in range(num_groups)])

In [ ]:
barWidth = 0.25
delta = 0.1
br1 = np.arange(num_groups)
br2 = [x + barWidth for x in br1]

plt.figure(figsize=(10, 7))
plt.bar(br1, ACI_coverage_rate, color = 'm', width = barWidth, edgecolor = 'gray', label = 'ACI')
plt.bar(br2, MVP_coverage_rate, color = 'c', width = barWidth, edgecolor = 'gray', label = 'MVP')
group_labels = [str(i) for i in range(1, num_groups+1)]
plt.xticks([r + barWidth for r in range(num_groups)], group_labels)
plt.axhline(y= 1 - delta, c = 'r', linewidth = 0.5)
plt.text(14, 1 - delta + 0.02, '  desired coverage')
plt.legend()
plt.ylim([0.0,1.2])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('Coverage on Noisy Groups: MVP vs ACI')
plt.show()

###### Experiment: Sorted Scores

In [ ]:
synthetic_scores = np.linspace(0, 0.5, num=len(prices))
plt.figure(figsize=(10, 7))
plt.plot(synthetic_scores)
plt.ylim([0, 1])
plt.xlabel('Days')
plt.ylabel('Scores')
plt.title('Sorted scores fed to MVP and ACI')
plt.show()

In [ ]:
# ACI

ACI_thresholds_sorted, ACI_alphas_sorted, ACI_miscoverage_rate_sorted = ACI_method(synthetic_scores)

plt.figure(figsize=(10, 7))
plt.hist(ACI_thresholds_sorted, bins = 40)
plt.xlabel('Threshold values (binned)')
plt.ylabel('No. times threshold value is predicted by ACI')
plt.title('Histogram of ACI thresholds on sorted data')
plt.show()

In [ ]:
# MVP

n_buckets = 40
T = len(synthetic_scores)
eta_sorted = np.sqrt(np.log(2 * 1 * n_buckets) / T)

MVP_sorted = MultiValidPrediction(n_buckets=n_buckets, eta=eta_sorted, normalize_by_counts=False)

for t in range(T):
    x_t = t
    score_sorted = synthetic_scores[t]

    thr_sorted = MVP_sorted.predict(x_t)
    MVP_sorted.update(x_t, thr_sorted, score_sorted)

thresholds_MVP_sorted, miscoverage_MVP_sorted, counts_MVP_sorted = MVP_sorted.coverage_stats(plot_thresholds=True, print_per_group_stats=True)

In [ ]:
print('Comparing predicted widths of ACI vs MVP:')

print('ACI widths: ', np.mean(np.array(ACI_thresholds_sorted)))
print('MVP widths: ', np.mean(np.array(thresholds_MVP_sorted)))

###### Experiment: Marginal coverage

In [ ]:
ACI_thrs_single, ACI_alphas_single, ACI_miscoverage_rate_single = ACI_method(scores, alpha=0.1, lookback=100, offset=10, gamma=0.005)

In [ ]:
# MVP

n_buckets = 40
T = len(scores)
eta = np.sqrt(np.log(2 * 1 * n_buckets) / T)

MVP_single = MultiValidPrediction(n_buckets=n_buckets, eta=eta, normalize_by_counts=False)

for t in range(T):
    x_t = t
    score = scores[t]

    thr = MVP_single.predict(x_t)
    MVP_single.update(x_t, thr, score)

thresholds_MVP_single, miscoverage_MVP_single, counts_MVP_single = MVP_single.coverage_stats(plot_thresholds=True, print_per_group_stats=True)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(ACI_thrs_single, label='ACI')
plt.plot(thresholds_MVP_single, label='MVP')
plt.legend(loc='lower right', shadow=True, fontsize='x-large')
plt.ylim([0.8, 1.02])
plt.xlabel('Days')
plt.ylabel('Thresholds')
plt.title('MVP vs ACI thresholds, AMD stock data')
plt.show()

###### Multigroup error bars for the Noisy Multigroup Coverage experiment (takes a long time to run due to trial number)

In [ ]:
num_groups = 20

scores_arr = []

for trial in range(20):
    returns_noisy = np.zeros(len(returns))
    std_returns = np.std(returns)
    for t in range(len(returns)):
        returns_noisy[t] = returns[t]
        for j in range(1, num_groups+1):
            if t % j == 0:
                returns_noisy[t] += std_returns*np.random.randn()

    volatility_noisy = [ret**2 for ret in returns_noisy]
    scores_noisy, predictions_noisy = garch_scores(volatility_noisy)

    scores_arr.append(scores_noisy)

import pickle

pickle_out = open('noisyscores0.pickle', 'wb')
pickle.dump(scores_arr, pickle_out)
pickle_out.close()

group_cvgs_ACI = []
group_cvgs_MVP = []

groups = []
for i in range(1, num_groups + 1):
    group = (lambda t, i=i: (t % i) == 0)
    groups.append(group)

for trial in range(20):
    scores_curr = scores_arr[trial]

    # ACI
    ACI_thrs, ACI_alphas, ACI_miscoverage_rate = ACI_method(scores_curr, alpha=0.1, lookback=100, offset=10, gamma=0.005)

    # ACI Coverage rates
    g_miscoverage = np.zeros(num_groups+1, dtype=int)
    g_counts = np.zeros(num_groups+1, dtype=int)

    thrs_seq = ACI_thrs
    for t in range(10, len(thrs_seq)):
        for j in range(1, num_groups+1):
            if t % j == 0:
                g_counts[j] += 1
                g_miscoverage[j] += int(scores_curr[t] > thrs_seq[t])

    ACI_coverage_rate = 1-np.array([g_miscoverage[j]/g_counts[j] for j in range(1, num_groups+1)])

    # MVP
    n_buckets = 40
    T = len(scores_curr)
    eta = np.sqrt(np.log(2 * len(groups) * n_buckets) / T)
    MVP_group = MultivalidPredictionIntervals(n_buckets=n_buckets, groups=groups, eta=eta)
    for t in range(T):
        x_t = t
        thr = MVP_group.predict(x_t)
        score = scores_curr[t]
        MVP_group.update(x_t, thr, score)

    # MVP Coverage rates
    thresholds_MVP_group, miscoverage_MVP_group, counts_MVP_group = MVP_group.coverage_stats(plot_thresholds=False, print_per_group_stats=False)
    MVP_coverage_rate = 1-np.array([np.sum(miscoverage_MVP_group[:, j])/np.sum(counts_MVP_group[:, j]) for j in range(num_groups)])

    # Add ACI and MVP coverage to array
    group_cvgs_ACI.append(ACI_coverage_rate)
    group_cvgs_MVP.append(MVP_coverage_rate)

cvgs_ACI = np.array(group_cvgs_ACI)
cvgs_MVP = np.array(group_cvgs_MVP)

# Error bars and medians
ACI_err_upper = [np.quantile(cvgs_ACI[:,group], 0.75) for group in range(num_groups)]
MVP_err_upper = [np.quantile(cvgs_MVP[:,group], 0.75) for group in range(num_groups)]

ACI_err_lower = [np.quantile(cvgs_ACI[:,group], 0.25) for group in range(num_groups)]
MVP_err_lower = [np.quantile(cvgs_MVP[:,group], 0.25) for group in range(num_groups)]

ACI_median    = [np.quantile(cvgs_ACI[:,group], 0.50) for group in range(num_groups)]
MVP_median    = [np.quantile(cvgs_MVP[:,group], 0.50) for group in range(num_groups)]

# Histogram

barWidth = 0.25
delta = 0.1
br1 = np.arange(num_groups)
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

plt.bar(br1, ACI_median, color = 'm', width = barWidth, edgecolor = 'gray', label = 'ACI')
plt.bar(br2, MVP_median, color = 'c', width = barWidth, edgecolor = 'gray', label = 'MVP')

for i, val in enumerate(br1):
    plt.vlines(x=val, ymin=ACI_err_lower[i], ymax=ACI_err_upper[i], color='black', linewidth=2)

for i, val in enumerate(br2):
    plt.vlines(x=val, ymin=MVP_err_lower[i], ymax=MVP_err_upper[i], color='black', linewidth=2)

group_labels = [str(i) for i in range(1, num_groups+1)]
plt.xticks([r + barWidth for r in range(num_groups)], group_labels)
plt.axhline(y= 1 - delta, c = 'r', linewidth = 0.5)
plt.text(14, 1 - delta + 0.02, '  desired coverage')
plt.legend()
plt.ylim([0.0,1.2])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.title('Coverage on Noisy Groups: MVP vs ACI')
plt.xlabel('Group number')
plt.ylabel('Coverage level')
plt.show()

# print maximum error bar width for ACI and the corresponding group
maxdist = 0
gr = -1
for group in range(num_groups):
    a = ACI_err_lower[group] 
    b = ACI_median[group]
    c = ACI_err_upper[group]
    if abs(a-c) > maxdist:
        maxdist = abs(a-c)
        gr = group
print(maxdist)
print(gr)

# print maximum error bar width for MVP and the corresponding group
maxdist = 0
gr = -1
for group in range(num_groups):
    a = MVP_err_lower[group] 
    b = MVP_median[group]
    c = MVP_err_upper[group]
    if abs(a-c) > maxdist:
        maxdist = abs(a-c)
        gr = group
print(maxdist)
print(gr)